# 코로나 데이터 웹크롤링 ~ 클린징

#### 웹 크롤링할 사이트 접속

In [ ]:
from selenium import webdriver
from bs4 import BeautifulSoup
import time

driver = webdriver.Chrome('./chromedriver')

url = 'https://www.bloomberg.com/graphics/covid-vaccine-tracker-global-distribution/'
driver.get(url)

#### 더보기 클릭하여 데이터 확보

In [15]:
# show more 버튼 클릭
sel_more1 = driver.find_element_by_xpath('//*[@id="dvz-table-global-vaccination"]/div[2]/div[2]/button')
sel_more1.click()
time.sleep(1)

In [16]:
# show more 버튼 한 번 더 클릭
sel_more2 = driver.find_element_by_xpath('//*[@id="dvz-table-global-vaccination"]/div[2]/div[2]/button')
sel_more2.click()


#### xpath 규칙 찾아서 국가명 크롤링

In [18]:
# Mainland China xpath : //*[@id="dvz-table-global-vaccination"]/div[2]/div[1]/table/tbody/tr[2]/td[1]
all_data = []

for i in range(1,7,1): # 중간중간에 6개의 백신정보가 섞여잇기 때문에 
    data_col = []
    xpath = '//*[@id="dvz-table-global-vaccination"]/div[2]/div[1]/table/tbody/tr/td[%s]' % str(i) 
    sel_data = driver.find_elements_by_xpath(xpath)
    
    for dat in sel_data:
        data_col.append(dat.text)
        
    print(data_col)
    all_data.append(data_col)

['Global Total', 'Mainland China', 'India', 'EU', 'U.S.', 'Brazil', 'Japan', 'Indonesia', 'Turkey', 'Germany', 'Mexico', 'France', 'U.K.', '', '', '', '', '', 'Russia', 'Italy', 'Pakistan', 'Spain', 'South Korea', 'Canada', '', '', '', '', '', '', '', '', '', '', '', '', '', 'Argentina', 'Thailand', 'Philippines', 'Iran', 'Malaysia', 'Saudi Arabia', 'Bangladesh', 'Morocco', 'Colombia', 'Vietnam', 'Poland', 'Chile', 'Australia', 'Sri Lanka', 'Peru', 'Cambodia', 'Netherlands', 'Ecuador', 'Cuba', 'UAE', 'Uzbekistan', 'Ukraine', 'South Africa', 'Belgium', 'Portugal', 'Egypt', 'Israel', 'Taiwan', 'Algeria', 'Venezuela', 'Kazakhstan', 'Sweden', 'Nepal', 'Greece', 'Dominican Republic', 'Czech Republic', 'Hungary', 'Austria', 'Switzerland', '', 'Romania', 'Singapore', 'Myanmar', 'Denmark', 'Hong Kong', 'Azerbaijan', 'Norway', 'Finland', 'Iraq', 'Tunisia', 'El Salvador', 'Ireland', 'Jordan', 'Guatemala', 'Bolivia', 'Nigeria', 'Serbia', 'Uruguay', 'Panama', 'Costa Rica', 'Zimbabwe', 'Honduras', 

#### 데이터 프레임 생성

In [49]:
import pandas as pd

pd.set_option("display.max_rows", 40)

dat_dict = {'국가':all_data[0],
            '백신접종수':all_data[1],
            'Enough_percent_people:':all_data[2],
            '1차접종':all_data[3],
            '2차접종':all_data[4],
            '일별접종수':all_data[5]
           }

dat_df = pd.DataFrame(dat_dict)
dat_df

,국가,백신접종수,Enough_percent_people:,1차접종,2차접종,일별접종수
0,Global Total,"6,199,443,468",–,–,–,"29,152,868"
1,Mainland China,"2,203,185,000",78.7,78.6,73.0,"3,171,286"
2,India,"876,538,633",32.0,46.7,16.8,"7,178,901"
3,EU,"564,029,956",63.5,67.3,64.7,"1,052,903"
4,U.S.,"391,152,574",61.1,64.4,55.8,"701,422"
...,...,...,...,...,...,...
221,Nauru,"14,863",57.2,58.6,55.8,4
222,St. Helena,"7,892",65.8,72.7,58.9,8
223,Falkland Islands,"4,407",73.5,87.7,59.2,202
224,Montserrat,"2,871",28.7,29.7,27.8,2


In [50]:
dat_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 226 entries, 0 to 225
Data columns (total 6 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   국가                      226 non-null    object
 1   백신접종수                   226 non-null    object
 2   Enough_percent_people:  226 non-null    object
 3   1차접종                    226 non-null    object
 4   2차접종                    226 non-null    object
 5   일별접종수                   226 non-null    object
dtypes: object(6)
memory usage: 10.7+ KB


In [51]:
dat_df['국가'].str.len()

0      12
1      14
2       5
3       2
4       4
       ..
221     5
222    10
223    16
224    10
225     7
Name: 국가, Length: 226, dtype: int64

#### 국가명 결측치값 확인 ( 문자열의 길이가 0 이면 값이 없는 것 )

In [52]:
dat_df['국가'].str.len().unique()

array([12, 14,  5,  2,  4,  6,  9,  7,  0,  8, 11, 10,  3, 18, 15, 22, 19,
       17, 21, 24, 16, 13, 30])

#### 국가명이 없는 경우 (U.K. 안에 London 같은것들 ) 데이터에서 삭제

In [53]:
print('전처리 전 : ',dat_df.shape)
dat_df = dat_df.loc[dat_df['국가'].str.len()>1,:]
print('전처리 후: ',dat_df.shape)
dat_df['국가'].str.len().unique()

전처리 전 :  (226, 6)
전처리 후:  (207, 6)


array([12, 14,  5,  2,  4,  6,  9,  7,  8, 11, 10,  3, 18, 15, 22, 19, 17,
       21, 24, 16, 13, 30])

#### 국가명이 없는 데이터 삭제후 인덱스값이 펑크남. -> 인덱스값 초기화

In [57]:
dat_df.head(40)

,국가,백신접종수,Enough_percent_people:,1차접종,2차접종,일별접종수
0,Global Total,"6,199,443,468",–,–,–,"29,152,868"
1,Mainland China,"2,203,185,000",78.7,78.6,73.0,"3,171,286"
2,India,"876,538,633",32.0,46.7,16.8,"7,178,901"
3,EU,"564,029,956",63.5,67.3,64.7,"1,052,903"
4,U.S.,"391,152,574",61.1,64.4,55.8,"701,422"
5,Brazil,"233,261,963",56.5,71.3,41.9,"1,317,163"
6,Japan,"161,057,159",63.8,69.5,58.2,"1,134,065"
7,Indonesia,"139,232,210",26.1,32.9,18.4,"1,916,012"
8,Turkey,"108,761,681",65.4,64.5,53.0,"381,129"
9,Germany,"107,197,993",64.5,67.9,64.2,"194,817"


In [58]:
#dat_df=dat_df.reindex()
dat_df=dat_df.reset_index()
dat_df.shape

(207, 7)

In [59]:
dat_df.head(40)

,index,국가,백신접종수,Enough_percent_people:,1차접종,2차접종,일별접종수
0,0,Global Total,"6,199,443,468",–,–,–,"29,152,868"
1,1,Mainland China,"2,203,185,000",78.7,78.6,73.0,"3,171,286"
2,2,India,"876,538,633",32.0,46.7,16.8,"7,178,901"
3,3,EU,"564,029,956",63.5,67.3,64.7,"1,052,903"
4,4,U.S.,"391,152,574",61.1,64.4,55.8,"701,422"
5,5,Brazil,"233,261,963",56.5,71.3,41.9,"1,317,163"
6,6,Japan,"161,057,159",63.8,69.5,58.2,"1,134,065"
7,7,Indonesia,"139,232,210",26.1,32.9,18.4,"1,916,012"
8,8,Turkey,"108,761,681",65.4,64.5,53.0,"381,129"
9,9,Germany,"107,197,993",64.5,67.9,64.2,"194,817"


#### 각 컬럼별 요소를 확인하면서 이상치값 존재 여부 확인 (각 리스트 = 컬럼 기준, 리스트 내 요소 = 컬럼 내 데이터 값)

In [62]:
col_all = dat_df.columns
for one in col_all:
    print(dat_df[one].unique())

[  0   1   2   3   4   5   6   7   8   9  10  11  12  18  19  20  21  22
  23  37  38  39  40  41  42  43  44  45  46  47  48  49  50  51  52  53
  54  55  56  57  58  59  60  61  62  63  64  65  66  67  68  69  70  71
  72  73  74  75  77  78  79  80  81  82  83  84  85  86  87  88  89  90
  91  92  93  94  95  96  97  98  99 100 101 102 103 104 105 106 107 108
 109 110 111 112 113 114 115 116 117 118 119 120 121 122 123 124 125 126
 127 128 129 130 131 132 133 134 135 136 137 138 139 140 141 142 143 144
 145 146 147 148 149 150 151 152 153 154 155 156 157 158 159 160 161 162
 163 164 165 166 167 168 169 170 171 172 173 174 175 176 177 178 179 180
 181 182 183 184 185 186 187 188 189 190 191 192 193 194 195 196 197 198
 199 200 201 202 203 204 205 206 207 208 209 210 211 212 213 214 215 216
 217 218 219 220 221 222 223 224 225]
['Global Total' 'Mainland China' 'India' 'EU' 'U.S.' 'Brazil' 'Japan'
 'Indonesia' 'Turkey' 'Germany' 'Mexico' 'France' 'U.K.' 'Russia' 'Italy'
 'Pakistan' 'Sp

#### 이상치값 '-'을 갖고 있는 데이터 확인

In [42]:
dat_df.loc[dat_df['백신접종수']=='–']

,국가,백신접종수,Enough_percent_people:,1차접종,2차접종,일별접종수
225,Eritrea,–,–,–,–,–


In [43]:
dat_df.loc[dat_df['일별접종수']=='–']

,국가,백신접종수,Enough_percent_people:,1차접종,2차접종,일별접종수
218,Turkmenistan,"41,993",0.4,0.6,0.2,–
225,Eritrea,–,–,–,–,–


#### 각 컬럼별 이상치 현황 확인

In [45]:
col_all = dat_df.columns
for one in col_all:
    print("col name : ", one)
    print( dat_df.loc[dat_df[one] == '–', one].count() )
    print("\n")

col name :  국가
0


col name :  백신접종수
1


col name :  Enough_percent_people:
5


col name :  1차접종
6


col name :  2차접종
7


col name :  일별접종수
2




#### 각 컬럼의 이상치값'-'를 -999와 0.05로 초기화

In [71]:
col_all = dat_df.columns
for one in col_all:
    print("col name : ", one)
    print( dat_df.loc[dat_df[one] == '–', one].count() )
    
    dat_df.loc[dat_df[one] == '–', one] = "-999"     # -값은 -999로 문자열로 
    dat_df.loc[dat_df[one] == '<0.1', one] = "0.05"  # < 가들어가있으면 0.05로 치환
    
    print("\n")

col name :  index
0


col name :  국가
0


col name :  백신접종수
1


col name :  Enough_percent_people:
5


col name :  1차접종
6


col name :  2차접종
7


col name :  일별접종수
2




#### 각 컬럼별 이상치값 변경 확인

In [74]:
col_all = dat_df.columns
for one in col_all:
    print("col name : ", one)
    print( dat_df.loc[dat_df[one] == '–', one].count() )
    print("\n")

col name :  index
0


col name :  국가
0


col name :  백신접종수
0


col name :  Enough_percent_people:
0


col name :  1차접종
0


col name :  2차접종
0


col name :  일별접종수
0




#### 데이터 전처리 -> 콤마, 없애기

In [75]:
dat_df['백신접종수'] = dat_df['백신접종수'].str.replace(',','')
dat_df['일별접종수'] = dat_df['일별접종수'].str.replace(',','')
dat_df.head()

,index,국가,백신접종수,Enough_percent_people:,1차접종,2차접종,일별접종수
0,0.0,Global Total,6199443468,-999,-999,-999,29152868
1,1.0,Mainland China,2203185000,78.7,78.6,73.0,3171286
2,2.0,India,876538633,32.0,46.7,16.8,7178901
3,3.0,EU,564029956,63.5,67.3,64.7,1052903
4,4.0,U.S.,391152574,61.1,64.4,55.8,701422


#### 결측치 존재 여부 확인

In [76]:
dat_df.isnull().sum()

index                     0
국가                        0
백신접종수                     0
Enough_percent_people:    0
1차접종                      0
2차접종                      0
일별접종수                     0
dtype: int64

#### 자료형 확인

In [77]:
dat_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 207 entries, 0 to 206
Data columns (total 7 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   index                   207 non-null    object
 1   국가                      207 non-null    object
 2   백신접종수                   207 non-null    object
 3   Enough_percent_people:  207 non-null    object
 4   1차접종                    207 non-null    object
 5   2차접종                    207 non-null    object
 6   일별접종수                   207 non-null    object
dtypes: object(7)
memory usage: 11.4+ KB


#### 숫자형 데이터가 object(문자열)로 되어있는 경우가 있다. 모두 숫자형 자료형으로 변경해주자

In [78]:
sel_col = dat_df.columns

for one in sel_col:
    print('col name : ', one)
    
    # 자료형 변환
    if one != '국가':
        print('자료형 변환 전 col type:', dat_df[one].dtype)
        dat_df[one] = dat_df[one].astype('float32')
        print('자료형 변환 후 col type:', dat_df[one].dtype)
    print()

col name :  index
자료형 변환 전 col type: object
자료형 변환 후 col type: float32

col name :  국가

col name :  백신접종수
자료형 변환 전 col type: object
자료형 변환 후 col type: float32

col name :  Enough_percent_people:
자료형 변환 전 col type: object
자료형 변환 후 col type: float32

col name :  1차접종
자료형 변환 전 col type: object
자료형 변환 후 col type: float32

col name :  2차접종
자료형 변환 전 col type: object
자료형 변환 후 col type: float32

col name :  일별접종수
자료형 변환 전 col type: object
자료형 변환 후 col type: float32



In [79]:
dat_df.head(30)

,index,국가,백신접종수,Enough_percent_people:,1차접종,2차접종,일별접종수
0,0.0,Global Total,6.199443e+09,-999.000000,-999.000000,-999.000000,29152868.0
1,1.0,Mainland China,2.203185e+09,78.699997,78.599998,73.000000,3171286.0
2,2.0,India,8.765386e+08,32.000000,46.700001,16.799999,7178901.0
3,3.0,EU,5.640300e+08,63.500000,67.300003,64.699997,1052903.0
4,4.0,U.S.,3.911526e+08,61.099998,64.400002,55.799999,701422.0
5,5.0,Brazil,2.332620e+08,56.500000,71.300003,41.900002,1317163.0
6,6.0,Japan,1.610572e+08,63.799999,69.500000,58.200001,1134065.0
7,7.0,Indonesia,1.392322e+08,26.100000,32.900002,18.400000,1916012.0
8,8.0,Turkey,1.087617e+08,65.400002,64.500000,53.000000,381129.0
9,9.0,Germany,1.071980e+08,64.500000,67.900002,64.199997,194817.0


#### 파일 저장하기

In [81]:
from datetime import datetime
import os

now = datetime.now()
file_make_time = "%04d%02d%02d_%02d" % (now.year, now.month, now.day, now.hour)

print( os.getcwd() )
path_dir = os.getcwd() + "/output_data/"  # window는 \\output_data\\ 이고 mac은 /output_data/이다.
path_file = path_dir + str(file_make_time)
print( path_dir,  path_file, sep="\n" )

dat_df.to_csv(path_file + "_vaccine_bloomberg.csv", index=False)
dat_df.to_excel(path_file + "_vaccine_bloomberg.xlsx", index=False)
os.listdir( path_dir )

/Users/works/Documents/Jupyter
/Users/works/Documents/Jupyter/output_data/
/Users/works/Documents/Jupyter/output_data/20210929_14


['20210929_14_vaccine_bloomberg.xlsx',
 '2021-09-28_corona.csv',
 '20210929_14_vaccine_bloomberg.csv',
 '2021-09-28_corona.xlsx']